# YT Sentinment Analysis

In [11]:
# Import the libraries you'll need here.
import pandas as pd
import numpy as np

# A basic visualization library
import matplotlib.pyplot as plt

# A great visualization library
import seaborn as sns
# This command tells python to use seaborn for its styling.
sns.set()

# Very important, this will make your charts appear in your notebook instead of in a new window.
%matplotlib inline

# Provides z-score helper function
import scipy.stats as stats

import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from wordcloud import WordCloud

import re, string

import demoji
from langdetect import detect

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Justin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Justin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Justin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data Cleaning

In [2]:
# Load Data set
df = pd.read_csv('data/UScomments.csv', error_bad_lines=False)
df.head()

b'Skipping line 41589: expected 4 fields, saw 11\nSkipping line 51628: expected 4 fields, saw 7\nSkipping line 114465: expected 4 fields, saw 5\n'
b'Skipping line 142496: expected 4 fields, saw 8\nSkipping line 189732: expected 4 fields, saw 6\nSkipping line 245218: expected 4 fields, saw 7\n'
b'Skipping line 388430: expected 4 fields, saw 5\n'
C:\Users\Justin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0


In [3]:
# Find null values
df.isnull().sum()

video_id         0
comment_text    25
likes            0
replies          0
dtype: int64

In [4]:
# Drop and check null values again
df = df.dropna()
df.isnull().sum()

video_id        0
comment_text    0
likes           0
replies         0
dtype: int64

In [5]:
# Find duplicates
print(df.duplicated().sum(), "\t")
df.duplicated()

212683 	


0         False
1         False
2         False
3         False
4         False
          ...  
691395     True
691396     True
691397     True
691398    False
691399     True
Length: 691375, dtype: bool

In [6]:
# Drop dupl. and check size of df
df = df.drop_duplicates()
df = df.reset_index(drop=True)
print(df.shape)
df.head(15)

(478692, 4)


,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0
5,XpVt6Z1Gjjo,#1 on trending AYYEEEEE,3,0
6,XpVt6Z1Gjjo,The end though 😭👍🏻❤️,4,0
7,XpVt6Z1Gjjo,#1 trending!!!!!!!!!,3,0
8,XpVt6Z1Gjjo,Happy one year vlogaversary,3,0
9,XpVt6Z1Gjjo,You and your shit brother may have single hand...,0,0


In [7]:
# Remove emojis from text
demoji.download_codes()
df['comments_cleaned'] = df['comment_text'].apply(lambda x: demoji.replace(x,""))

... OK (Got response in 0.13 seconds)
Writing emoji data to C:\Users\Justin\.demoji\codes.json ...
... OK


In [8]:
df.head(15)

,video_id,comment_text,likes,replies,comments_cleaned
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0,Logan Paul it's yo big day
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0,I've been following you from the start of your...
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0,Say hi to Kong and maverick for me
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0,MY FAN . attendance
4,XpVt6Z1Gjjo,trending 😉,3,0,trending
5,XpVt6Z1Gjjo,#1 on trending AYYEEEEE,3,0,#1 on trending AYYEEEEE
6,XpVt6Z1Gjjo,The end though 😭👍🏻❤️,4,0,The end though
7,XpVt6Z1Gjjo,#1 trending!!!!!!!!!,3,0,#1 trending!!!!!!!!!
8,XpVt6Z1Gjjo,Happy one year vlogaversary,3,0,Happy one year vlogaversary
9,XpVt6Z1Gjjo,You and your shit brother may have single hand...,0,0,You and your shit brother may have single hand...


In [9]:
#Language Detection (only retrieve english comments)
#Add 'language' col
df['lang'] = 0

In [12]:
count = 0
for i in range(0,len(df)):
    temp = df['comments_cleaned'].iloc[i]
    count += 1
    try:
        df['lang'].iloc[i] = detect(temp)
    except:
        df['lang'].iloc[i] = "error"

C:\Users\Justin\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
df.head()

,video_id,comment_text,likes,replies,comments_cleaned,lang
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0,Logan Paul it's yo big day,tl
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0,I've been following you from the start of your...,en
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0,Say hi to Kong and maverick for me,en
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0,MY FAN . attendance,it
4,XpVt6Z1Gjjo,trending 😉,3,0,trending,da


In [19]:
#make seperate df with only english comments
english_comm = df[df['lang'] == 'en']

In [17]:
english_comm.shape

(384078, 6)

In [21]:
english_comm.to_csv("data/US_english_comments.csv",index=False)

In [23]:
english_comm.head()

,video_id,comment_text,likes,replies,comments_cleaned,lang
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0,I've been following you from the start of your...,en
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0,Say hi to Kong and maverick for me,en
5,XpVt6Z1Gjjo,#1 on trending AYYEEEEE,3,0,#1 on trending AYYEEEEE,en
6,XpVt6Z1Gjjo,The end though 😭👍🏻❤️,4,0,The end though,en
8,XpVt6Z1Gjjo,Happy one year vlogaversary,3,0,Happy one year vlogaversary,en
